In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log(
        (responses_column + epsilon)
        / (answers_column + epsilon)
    )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):  # answers - правильные ответы
    log_accuracy_ratio_mean = np.array(
        [
            get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon),
        ]
    ).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error.round(
        decimals=2
    )

In [0]:
import pandas as pd
import numpy as np

In [0]:
DF = "drive/My Drive/VK_Cup_#2/Data/top_validate.tsv"
VALIDATE_ANSWERS = "drive/My Drive/VK_Cup_#2/Data/validate_answers.tsv"

In [0]:
Y = pd.read_csv(VALIDATE_ANSWERS, sep='\t')

In [6]:
Y.head()

,at_least_one,at_least_two,at_least_three
0,0.0430,0.0152,0.0073
1,0.0130,0.0000,0.0000
2,0.0878,0.0135,0.0000
3,0.2295,0.1295,0.0727
4,0.3963,0.2785,0.2270


In [0]:
Y_1 = Y['at_least_one'].copy()
Y_2 = Y['at_least_two'].copy()
Y_3 = Y['at_least_three'].copy()

In [8]:
Y_1

0       0.0430
1       0.0130
2       0.0878
3       0.2295
4       0.3963
         ...  
1003    0.0679
1004    0.0165
1005    0.0071
1006    0.3669
1007    0.0710
Name: at_least_one, Length: 1008, dtype: float64

In [9]:
X = pd.read_csv(DF).astype('float32')
X.head()

,cpm,hour_start,hour_end,audience_size,time_interval,hour_start_time,hour_end_time,time_interval_time,middle_time,part_day,people_partday_platform,percent_people_partday_platform,people_patday,percent_people_patday,percent_people_partday_platformy,count_unique_cities,number_of_sex1,number_of_sex2,number_of_age1,number_of_age2,number_of_age3,number_of_age4,number_of_age5,number_of_age6,percentage_sex1,percentage_sex2,percentage_age1,percentage_age2,percentage_age3,percentage_age4,percentage_age5,percentage_age6,mean_cpm_in_partday_and_publishers,number_of_publishers,publisher_1,publisher_2,publisher_3,publisher_4,publisher_5,publisher_6,publisher_7,publisher_8,publisher_9,publisher_10,publisher_11,publisher_12,publisher_13,publisher_14,publisher_15,publisher_16,publisher_17,publisher_18,publisher_19,publisher_20,publisher_21,number_who_use_that_publishers,percentage_publishers_users
0,220.0,1058.0,1153.0,1906.0,95.0,2.0,1.0,23.0,1106.0,3.0,355.0,0.21,1457.0,0.76,0.19,7.0,1025.0,881.0,470.0,776.0,427.0,179.0,19.0,35.0,0.54,0.46,0.25,0.41,0.22,0.09,0.01,0.02,257.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,406.0,0.21
1,312.0,1295.0,1301.0,1380.0,6.0,23.0,5.0,6.0,1298.0,3.0,163.0,0.06,1373.0,0.99,0.12,372.0,735.0,645.0,428.0,508.0,287.0,106.0,16.0,35.0,0.53,0.47,0.31,0.37,0.21,0.08,0.01,0.03,204.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,163.0,0.12
2,70.0,1229.0,1249.0,888.0,20.0,5.0,1.0,20.0,1239.0,3.0,607.0,0.81,661.0,0.74,0.68,280.0,473.0,415.0,234.0,330.0,190.0,107.0,11.0,16.0,0.53,0.47,0.26,0.37,0.21,0.12,0.01,0.02,194.0,6.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,724.0,0.82
3,240.0,1295.0,1377.0,440.0,82.0,23.0,9.0,10.0,1336.0,3.0,203.0,0.44,334.0,0.76,0.46,7.0,240.0,200.0,119.0,187.0,78.0,43.0,6.0,7.0,0.55,0.45,0.27,0.42,0.18,0.10,0.01,0.02,186.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,239.0,0.54
4,262.0,752.0,990.0,1476.0,238.0,8.0,6.0,22.0,871.0,2.0,768.0,0.71,899.0,0.61,0.52,416.0,903.0,572.0,0.0,0.0,215.0,1261.0,0.0,0.0,0.61,0.39,0.00,0.00,0.15,0.85,0.00,0.00,180.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1076.0,0.73


In [10]:
X.columns

Index(['cpm', 'hour_start', 'hour_end', 'audience_size', 'time_interval',
       'hour_start_time', 'hour_end_time', 'time_interval_time', 'middle_time',
       'part_day', 'people_partday_platform',
       'percent_people_partday_platform', 'people_patday',
       'percent_people_patday', 'percent_people_partday_platformy',
       'count_unique_cities', 'number_of_sex1', 'number_of_sex2',
       'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4',
       'number_of_age5', 'number_of_age6', 'percentage_sex1',
       'percentage_sex2', 'percentage_age1', 'percentage_age2',
       'percentage_age3', 'percentage_age4', 'percentage_age5',
       'percentage_age6', 'mean_cpm_in_partday_and_publishers',
       'number_of_publishers', 'publisher_1', 'publisher_2', 'publisher_3',
       'publisher_4', 'publisher_5', 'publisher_6', 'publisher_7',
       'publisher_8', 'publisher_9', 'publisher_10', 'publisher_11',
       'publisher_12', 'publisher_13', 'publisher_14', 'publis

In [0]:
X = X.drop(['hour_start', 'hour_end'], axis=1)

In [0]:
x_lim=int(len(X)*0.80)

X_train=X[0:x_lim].copy()
Y_train_1=Y_1[0:x_lim]
Y_train_2=Y_2[0:x_lim]
Y_train_3=Y_3[0:x_lim]

X_test=X[x_lim:].copy()
Y_test_1=Y_1[x_lim:]
Y_test_2=Y_2[x_lim:]
Y_test_3=Y_3[x_lim:]

In [13]:
Y_train_1.shape, Y_test_1.shape

((806,), (202,))

In [0]:
Y_test = pd.concat([Y_test_1, Y_test_2, Y_test_3], axis=1)
Y_test = pd.DataFrame(np.array(Y_test), columns=['at_least_one', 'at_least_two', 'at_least_three']).astype('float32')

In [15]:
Y_test.head()

,at_least_one,at_least_two,at_least_three
0,0.0163,0.0041,0.0014
1,0.0201,0.0000,0.0000
2,0.1761,0.0954,0.0578
3,0.0211,0.0105,0.0039
4,0.1165,0.0059,0.0000


In [16]:
from sklearn.preprocessing import MinMaxScaler

mms=MinMaxScaler()
X_train_mms=mms.fit_transform(X_train)
X_test_mms=mms.fit_transform(X_test)

print('Минимум и максимум обучающего набора: ', X_train_mms.min(), X_train_mms.max())

Минимум и максимум обучающего набора:  0.0 1.0000001


In [17]:
from sklearn.preprocessing import StandardScaler

mms=StandardScaler()
X_train_mms=mms.fit_transform(X_train)
X_test_mms=mms.fit_transform(X_test)

print('Минимум и максимум обучающего набора: ', X_train_mms.min(), X_train_mms.max())

Минимум и максимум обучающего набора:  -3.825924 9.062524


In [0]:
X_train_mms = X_train
X_test_mms = X_test

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf2 = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf3 = RandomForestRegressor(n_estimators = 1000, random_state = 42)

rf1.fit(X_train_mms, Y_train_1)
predictions_1 = rf1.predict(X_test_mms)
rf2.fit(X_train_mms, Y_train_2)
predictions_2 = rf2.predict(X_test_mms)
rf3.fit(X_train_mms, Y_train_3)
predictions_3 = rf3.predict(X_test_mms)

In [0]:
Y_pesponse = pd.DataFrame([predictions_1, predictions_2, predictions_3]).astype('float32').T
Y_pesponse.columns=['at_least_one', 'at_least_two', 'at_least_three']

In [21]:
get_smoothed_mean_log_accuracy_ratio(Y_test, Y_pesponse)

37.41